In [1]:
import numpy as np 
import re 
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer , sent_tokenize, word_tokenize
import os
from bs4 import BeautifulSoup
import requests

In [2]:
#reading the text files containing positive words
positiveWordsList = open('positive-words.txt').read()

#reading the text files containing negative words
nagitiveWordsList = open('negative-words.txt').read()

#reading the text files form the folder which contains all the stop words
def load_stop_words(folder):
    stop_words = set()
    
    for file in os.listdir(folder):
        file_path = os.path.join(folder, file)
        
        if file.endswith('.txt'):
            try:
                # UTF-8 first
                with open(file_path, 'r', encoding='utf-8') as file:
                    for line in file:
                        stop_words.add(line.strip().lower())
            except UnicodeDecodeError:
                # If UTF-8 fails,'latin-1' encoding
                with open(file_path, 'r', encoding='latin-1') as file:
                    for line in file:
                        stop_words.add(line.strip().lower())
                    
    return stop_words

folder = 'StopWords'

stopWordList = load_stop_words(folder)

In [3]:
#read the file containing the input URLS
input = pd.read_excel("Input.xlsx")
input

#getting the titles of the URLS
def get_title(Url):
    titles=[]
    for i in range(len(Url)):
        heading=Url[i]
        clean=heading[heading.index("m/")+2:-1].replace('-',' ')
        titles.append(clean)
    return titles

url=input["URL"]
URL_title=get_title(url)
URL_title

['ai and ml based youtube analytics and content creation tool for optimizing subscriber engagement and content strategy',
 'enhancing front end features and functionality for improved user experience and dashboard accuracy in partner hospital application',
 'roas dashboard for campaign wise google ads budget tracking using google ads ap',
 'efficient processing and analysis of financial data from pdf files addressing formatting inconsistencies and ensuring data integrity for a toyota dealership management firm',
 'development of ea robot for automated trading',
 'ai and ml based youtube analytics and content creation tool for optimizing subscriber engagement and content strategy',
 'enhancing front end features and functionality for improved user experience and dashboard accuracy in partner hospital application',
 'roas dashboard for campaign wise google ads budget tracking using google ads ap',
 'efficient processing and analysis of financial data from pdf files addressing formatting 

In [4]:
#printing the total numbers of URLS
print(len(URL_title))

147


In [5]:
# positive words
with open('positive-words.txt','r') as posfile:
    positivewords=posfile.read().lower()
positiveWordList=positivewords.split('\n')


# negative words
with open('negative-words.txt' ,'r') as negfile:
    negativeword=negfile.read().lower()
negativeWordList=negativeword.split('\n')

In [6]:
URLS = input ["URL"]
URLS

0      https://insights.blackcoffer.com/ai-and-ml-bas...
1      https://insights.blackcoffer.com/enhancing-fro...
2      https://insights.blackcoffer.com/roas-dashboar...
3      https://insights.blackcoffer.com/efficient-pro...
4      https://insights.blackcoffer.com/development-o...
                             ...                        
142    https://insights.blackcoffer.com/population-an...
143    https://insights.blackcoffer.com/google-lsa-ap...
144    https://insights.blackcoffer.com/healthcare-da...
145    https://insights.blackcoffer.com/budget-sales-...
146    https://insights.blackcoffer.com/amazon-buy-bo...
Name: URL, Length: 147, dtype: object

In [7]:
data =[]
for url in URLS:

  page=requests.get(url)  
  soup = BeautifulSoup(page.text , 'html.parser')
  #get title
  title = soup.find("h1",class_="entry-title").get_text()
  print(title)
  #get the text corrosponding the title 
  text = soup.find(class_="td-post-content tagdiv-type").get_text()
  # print(text)

  # removing the space from the front and back
  lines=(line.strip() for line in text.splitlines())

  # converting into one huge chunk
  chunks=(phrase.strip() for line in lines for phrase in line.split("  "))

  # drop blank lines
  text1= '\n'.join(chunk for chunk in chunks if chunk)
  data.append(text1)

AI and ML-Based YouTube Analytics and Content Creation Tool for Optimizing Subscriber Engagement and Content Strategy
Enhancing Front-End Features and Functionality for Improved User Experience and Dashboard Accuracy in Partner Hospital Application
ROAS Dashboard for Campaign-Wise Google Ads Budget Tracking Using Google Ads AP
Efficient Processing and Analysis of Financial Data from PDF Files: Addressing Formatting Inconsistencies and Ensuring Data Integrity for a Toyota Dealership Management Firm
Development of EA Robot for Automated Trading
AI and ML-Based YouTube Analytics and Content Creation Tool for Optimizing Subscriber Engagement and Content Strategy
Enhancing Front-End Features and Functionality for Improved User Experience and Dashboard Accuracy in Partner Hospital Application
ROAS Dashboard for Campaign-Wise Google Ads Budget Tracking Using Google Ads AP
Efficient Processing and Analysis of Financial Data from PDF Files: Addressing Formatting Inconsistencies and Ensuring Dat

Big Data solution to an online multivendor marketplace eCommerce business
Creating a custom report and dashboard using the data got from Atera API
Azure Data Lake and Power BI Dashboard
Google Data Studio Pipeline with GCP/MySQL
QuickBooks dashboard to find patterns in finance, sales, and forecasts
Marketing, sales, and financial data business dashboard (Wink Report)
React Native Apps in the Development Portfolio
A Leading Law Firm in the USA, Website SEO & Optimization
A Leading Hospitality Firm in the USA, Website SEO & Optimization
A Leading Firm in the USA, Website SEO & Optimization
A Leading Musical Instrumental, Website SEO & Optimization
A Leading Firm in the USA, SEO and Website Optimization
Immigration Datawarehouse & AI-based recommendations
Lipsync Automation for Celebrities and Influencers
Key Audit Matters Predictive Modeling
Splitting of Songs into its Vocals and Instrumental
AI and ML technologies to Evaluate Learning Assessments
Datawarehouse, and Recommendations Engin

In [8]:
print(len(data))

147


In [9]:
cleaned_data = [text.replace('\xa0', ' ') for text in data]

In [10]:
#data frame and all the columns of the text
df = pd.DataFrame({'title':URL_title,'descp': cleaned_data})

In [11]:
#tokenizeing module and filtering tokens using stop words list, removing punctuations
def tokenizer(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = list(filter(lambda token: token not in stopWordList, tokens))
    return filtered_words

In [12]:
# calculating the positive score of the text
def positive_score (text):
    posword=0
    tokenphrase = tokenizer(text)
    for word in tokenphrase :
        if word in positiveWordList:
            posword+=1 
    return posword 
df["positive_score"] = df["descp"].apply (positive_score)

In [13]:
# calculating the negative score of the text
def negative_score (text):
    negword=0
    tokenphrase = tokenizer(text)
    for word in tokenphrase :
        if word in negativeWordList :
            negword +=1
    return negword
df["negative_score"] = df["descp"].apply (negative_score)

In [14]:
# calculating the polarity score of the text
def polarity_score (positive_score , negative_score) :
  return (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
df["polarity_score"] = np.vectorize(polarity_score)(df['positive_score'],df['negative_score'])

In [15]:
# calculating the total word count of the text
def total_word_count(text):
    tokens = tokenizer(text)
    return len(tokens)
df["total word count"] = df["descp"].apply (total_word_count)

In [16]:
# # calculating the number of complex words of the text
def complex_word_count(text):
    tokens = tokenizer(text)
    complexWord = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    return complexWord
df["complex_word_count"] = df["descp"].apply (complex_word_count)

# calculating the percentage of complex words of the text
def percentage_complex_word(text):
    tokens = tokenizer(text)
    complexWord = 0
    complex_word_percentage = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            
            if(vowels > 2):
                complexWord += 1
    if len(tokens) != 0:
        complex_word_percentage = complexWord/len(tokens)
    
    return complex_word_percentage
df["percentage_complex_word"] = df["descp"].apply (percentage_complex_word)

In [17]:
# calculating the Average_Words_Per_Sentence of the text
def Average_Words_Per_Sentence (text,word_count):
    SentenceCount = len (sent_tokenize(text))
    if SentenceCount > 0 : Average_Words_Per_Sentence = word_count / SentenceCount
    avg = Average_Words_Per_Sentence
    return round(avg)
df["Average_Words_Per_Sentence"] =np.vectorize(Average_Words_Per_Sentence)(df["descp"],df["total word count"])

# calculating the Average_Word_Length of the text
def Average_Word_Length(text,word_count):
    CharCount=len(text)
    if CharCount > 0 : Average_Words_Length = CharCount /word_count 
    avg = Average_Words_Length
    return avg
df["Average_Word_Length"] =  np.vectorize(Average_Word_Length)(df["descp"],df["total word count"])

In [18]:
# calculating the fog index of the text
def fog_index(averageSentenceLength, percentageComplexWord):
    fogIndex = 0.4 * (averageSentenceLength + percentageComplexWord)
    return fogIndex
df["Fog_Index"] = np.vectorize(fog_index)(df['Average_Words_Per_Sentence'],df['percentage_complex_word'])

# calculating the Subjectivity Score of the text
def Subjectivity_Score (positive_score , negative_score , word_count) :
  return (positive_score + negative_score) / ((word_count) + 0.000001)
df["Subjectivity_Score"] = np.vectorize(Subjectivity_Score)(df['positive_score'],df['negative_score'],df["total word count"])

In [19]:
# calculating the Syllable_Per_Word of the text
def Syllable_Per_Word(text,word_count):
    tokens = tokenizer(text)
    syllable=0
    words='aeiou'
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if w.lower() in words:
                    vowels += 1
                    syllable += 1
    average = syllable / word_count
    return average
df["Syllable_Per_Word"] = np.vectorize(Syllable_Per_Word)(df["descp"],df["total word count"])

In [20]:
nltk.download('punkt')
def tokenizer(text):
    return word_tokenize(text)
def Personal_Pronouns(text):
    check = ["i", "we", "ours", "my", "us"]
    token = tokenizer(text)
    pp = 0
    # Loop through the tokens and check for matches with the pronoun list
    for word in token:
        cleaned_word = word.strip('.,!?()[]{}":;').lower()
        if cleaned_word in check:
            pp += 1
    return pp
df["Personal_Pronouns"] = np.vectorize(Personal_Pronouns)(cleaned_data)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91905\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
df

,title,descp,positive_score,negative_score,polarity_score,total word count,complex_word_count,percentage_complex_word,Average_Words_Per_Sentence,Average_Word_Length,Fog_Index,Subjectivity_Score,Syllable_Per_Word,Personal_Pronouns
0,ai and ml based youtube analytics and content ...,Client BackgroundClient: A leading IT & tech f...,3,0,1.000000,174,76,0.436782,35,10.568966,14.174713,0.017241,2.327586,1
1,enhancing front end features and functionality...,Client BackgroundClient: A leading hospital ch...,10,7,0.176471,394,151,0.383249,28,13.362944,11.353299,0.043147,2.200508,7
2,roas dashboard for campaign wise google ads bu...,Client BackgroundClient: A leading IT & tech f...,10,3,0.538461,288,117,0.406250,18,9.829861,7.362500,0.045139,2.256944,1
3,efficient processing and analysis of financial...,Client BackgroundClient: A leading automobile ...,21,10,0.354839,397,198,0.498741,50,11.017632,20.199496,0.078086,2.390428,4
4,development of ea robot for automated trading,Objective:The goal of this project is to build...,9,3,0.500000,467,163,0.349036,13,10.113490,5.339615,0.025696,2.239829,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,population and community survey of america,Client BackgroundClient: A leading marketing f...,12,8,0.200000,556,182,0.327338,46,10.534173,18.530935,0.035971,2.048561,2
143,google lsa api data automation and dashboarding,Client BackgroundClient: A leading marketing f...,20,16,0.111111,784,319,0.406888,25,11.235969,10.162755,0.045918,2.359694,6
144,healthcare data analysis,Client BackgroundClient: A leading healthcare ...,8,10,-0.111111,195,71,0.364103,24,12.620513,9.745641,0.092308,2.256410,13
145,budget sales kpi dashboard using power bi,Project DescriptionWeekly Data – clustered bar...,0,0,0.000000,71,18,0.253521,71,11.859155,28.501408,0.000000,2.464789,0


In [22]:
#droping the column which is not required
final = df.drop("descp" , 1)
final 

C:\Users\91905\AppData\Local\Temp\ipykernel_4636\638239199.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  final = df.drop("descp" , 1)


,title,positive_score,negative_score,polarity_score,total word count,complex_word_count,percentage_complex_word,Average_Words_Per_Sentence,Average_Word_Length,Fog_Index,Subjectivity_Score,Syllable_Per_Word,Personal_Pronouns
0,ai and ml based youtube analytics and content ...,3,0,1.000000,174,76,0.436782,35,10.568966,14.174713,0.017241,2.327586,1
1,enhancing front end features and functionality...,10,7,0.176471,394,151,0.383249,28,13.362944,11.353299,0.043147,2.200508,7
2,roas dashboard for campaign wise google ads bu...,10,3,0.538461,288,117,0.406250,18,9.829861,7.362500,0.045139,2.256944,1
3,efficient processing and analysis of financial...,21,10,0.354839,397,198,0.498741,50,11.017632,20.199496,0.078086,2.390428,4
4,development of ea robot for automated trading,9,3,0.500000,467,163,0.349036,13,10.113490,5.339615,0.025696,2.239829,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,population and community survey of america,12,8,0.200000,556,182,0.327338,46,10.534173,18.530935,0.035971,2.048561,2
143,google lsa api data automation and dashboarding,20,16,0.111111,784,319,0.406888,25,11.235969,10.162755,0.045918,2.359694,6
144,healthcare data analysis,8,10,-0.111111,195,71,0.364103,24,12.620513,9.745641,0.092308,2.256410,13
145,budget sales kpi dashboard using power bi,0,0,0.000000,71,18,0.253521,71,11.859155,28.501408,0.000000,2.464789,0


In [24]:
final.to_excel('Output_Data_Structure.xlsx')